In [ ]:
# Instalação de Bibliotecas
pip install pandas numpy matplotlib scikit-learn tensorflow keras seaborn

In [1]:
# Importação de Bibliotecas

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.neural_network import MLPClassifier

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout, SpatialDropout1D, SimpleRNN, Embedding
from keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# Carregar os dados
df = pd.read_csv("C:/Users/ana_v/OneDrive/Documentos/Repositórios/GSE25066.csv", low_memory=False)

# Obtém os valores únicos na coluna 'Type'
unique_types = df['type'].unique()

# Cria um dicionário mapeando cada tipo único para um número
type_to_numeric = {type_name: index for index, type_name in enumerate(unique_types)}

# Aplica a substituição usando o método map
df['type'] = df['type'].map(type_to_numeric)

# Armazena a coluna 'type' para adicioná-la de volta posteriormente
type_column = df['type']

# Prepara o DataFrame para normalização (remover colunas desnecessárias)
df_num = df.drop(columns=["sample", "type"])

# Normalizar os dados
scaler = StandardScaler()
dados_normalizados = scaler.fit_transform(df_num)

# Aplicação do PCA
pca = PCA(n_components=0.8)  
pca.fit(dados_normalizados)
dados_pca = pca.transform(dados_normalizados)

In [3]:
# Fixar a seed para garantir a reprodutibilidade
seed = 1

# Divisão de treino e teste com random_state
X_train, X_test, y_train, y_test = train_test_split(dados_pca, df['type'], test_size=0.2, random_state=seed)

In [4]:
# Regressão Logística
lr_model = LogisticRegression(max_iter=6500) 
lr_model.fit(X_train, y_train) 
lr_predictions_train = lr_model.predict(X_train)  
lr_predictions_test = lr_model.predict(X_test) 

# Avaliação Regressão Logística
lr_accuracy_train = accuracy_score(y_train, lr_predictions_train) 
lr_accuracy_test = accuracy_score(y_test, lr_predictions_test)  
lr_report = classification_report(y_test, lr_predictions_test, zero_division=1) 

# Criar um objeto de validação cruzada
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# Substituir a divisão de treino e teste pelo código de validação cruzada
scores = cross_val_score(lr_model, dados_pca, df['type'], cv=cv, scoring='accuracy')

# Imprimir os resultados
print(f'Regressão Logística - Acurácia (Teste): {lr_accuracy_test}')
print(f'Acurácia média na validação cruzada: {scores.mean()}')
print(f'Classification Report:\n{lr_report}')
print("Matriz de Confusão:\n", confusion_matrix(y_test, lr_predictions_test))


Regressão Logística - Acurácia (Teste): 0.803921568627451
Acurácia média na validação cruzada: 0.7894901960784313
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.79      0.81        34
           1       0.50      0.62      0.56         8
           2       1.00      0.90      0.95        40
           3       0.62      0.77      0.69        13
           4       0.57      0.57      0.57         7

    accuracy                           0.80       102
   macro avg       0.70      0.73      0.71       102
weighted avg       0.82      0.80      0.81       102

Matriz de Confusão:
 [[27  2  0  4  1]
 [ 1  5  0  0  2]
 [ 1  3 36  0  0]
 [ 3  0  0 10  0]
 [ 1  0  0  2  4]]


In [7]:
# Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)  
rf_predictions_train = rf_model.predict(X_train)  
rf_predictions_test = rf_model.predict(X_test) 

# Avaliação Random Forest
rf_accuracy_train = accuracy_score(y_train, rf_predictions_train) 
rf_accuracy_test = accuracy_score(y_test, rf_predictions_test) 
rf_report = classification_report(y_test, rf_predictions_test, 
                                  zero_division=1) 

# Criar um objeto de validação cruzada
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# Substituir a divisão de treino e teste pelo código de validação cruzada
scores_rf = cross_val_score(rf_model, dados_pca, df['type'], cv=cv, scoring='accuracy')

# Imprimir os resultados
print(f'Regressão Logística - Acurácia (Teste): {rf_accuracy_test}')
print(f'Acurácia média na validação cruzada: {scores_rf.mean()}')
print(f'Classification Report:\n{rf_report}')
print("Matriz de Confusão:\n", confusion_matrix(y_test, rf_predictions_test))


Regressão Logística - Acurácia (Teste): 0.6862745098039216
Acurácia média na validação cruzada: 0.6298823529411763
Classification Report:
              precision    recall  f1-score   support

           0       0.60      0.88      0.71        34
           1       1.00      0.00      0.00         8
           2       0.78      0.95      0.85        40
           3       0.67      0.15      0.25        13
           4       1.00      0.00      0.00         7

    accuracy                           0.69       102
   macro avg       0.81      0.40      0.36       102
weighted avg       0.74      0.69      0.60       102

Matriz de Confusão:
 [[30  0  4  0  0]
 [ 6  0  2  0  0]
 [ 2  0 38  0  0]
 [ 9  0  2  2  0]
 [ 3  0  3  1  0]]


In [8]:
# SVM
svm_model = SVC()
svm_model.fit(X_train, y_train)  
svm_predictions_train = svm_model.predict(X_train)  
svm_predictions_test = svm_model.predict(X_test) 

# Avaliação SVM
svm_accuracy_train = accuracy_score(y_train, svm_predictions_train)  
svm_accuracy_test = accuracy_score(y_test, svm_predictions_test)  
svm_report = classification_report(y_test, svm_predictions_test, zero_division=1)  

# Criar um objeto de validação cruzada
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# Substituir a divisão de treino e teste pelo código de validação cruzada
scores_svm = cross_val_score(svm_model, dados_pca, df['type'], cv=cv, 
                         scoring='accuracy')

# Imprimir os resultados
print(f'Regressão Logística - Acurácia (Teste): {svm_accuracy_test}')
print(f'Acurácia média na validação cruzada: {scores_svm.mean()}')
print(f'Classification Report:\n{lr_report}')
print("Matriz de Confusão:\n", confusion_matrix(y_test, svm_predictions_test))


Regressão Logística - Acurácia (Teste): 0.7647058823529411
Acurácia média na validação cruzada: 0.7658431372549019
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.79      0.81        34
           1       0.50      0.62      0.56         8
           2       1.00      0.90      0.95        40
           3       0.62      0.77      0.69        13
           4       0.57      0.57      0.57         7

    accuracy                           0.80       102
   macro avg       0.70      0.73      0.71       102
weighted avg       0.82      0.80      0.81       102

Matriz de Confusão:
 [[32  0  0  2  0]
 [ 4  3  0  0  1]
 [ 2  0 37  0  1]
 [ 9  0  0  4  0]
 [ 2  0  1  2  2]]


In [9]:
# Criação e treinamento do MLP com 4 camadas ocultas
mlp_model = MLPClassifier(hidden_layer_sizes=(100, 75, 50, 25), max_iter=500, random_state=seed)
mlp_model.fit(X_train, y_train)

# Predições no conjunto de treino e teste
mlp_predictions_train = mlp_model.predict(X_train)
mlp_predictions_test = mlp_model.predict(X_test)

# Avaliação MLP
mlp_accuracy_train = accuracy_score(y_train, mlp_predictions_train)
mlp_accuracy_test = accuracy_score(y_test, mlp_predictions_test)
mlp_report = classification_report(y_test, mlp_predictions_test, zero_division=1)

# Criar um objeto de validação cruzada
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# Substituir a divisão de treino e teste pelo código de validação cruzada
mlp_scores = cross_val_score(mlp_model, dados_pca, df['type'], cv=cv, scoring='accuracy')

# Exibindo resultados do MLP
print(f'MLP - Acurácia (Teste): {mlp_accuracy_test}')
print(f'Acurácia média na validação cruzada (MLP): {mlp_scores.mean()}')
print(f'Classification Report MLP:\n{mlp_report}')
print("Matriz de Confusão:\n", confusion_matrix(y_test, mlp_predictions_test))


MLP - Acurácia (Teste): 0.6862745098039216
Acurácia média na validação cruzada (MLP): 0.6988235294117647
Classification Report MLP:
              precision    recall  f1-score   support

           0       0.65      0.71      0.68        34
           1       0.60      0.38      0.46         8
           2       0.84      0.90      0.87        40
           3       0.33      0.31      0.32        13
           4       0.60      0.43      0.50         7

    accuracy                           0.69       102
   macro avg       0.60      0.54      0.57       102
weighted avg       0.68      0.69      0.68       102

Matriz de Confusão:
 [[24  1  2  7  0]
 [ 2  3  1  1  1]
 [ 3  1 36  0  0]
 [ 6  0  2  4  1]
 [ 2  0  2  0  3]]


In [11]:
np.random.seed(seed)
tf.random.set_seed(seed)

unique_types = df['type'].unique()
num_classes = len(unique_types)
target_names = [str(cls) for cls in unique_types]

X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

# Função para criar o modelo CNN
def create_cnn_model():
    model = Sequential([
        Conv1D(32, 3, activation='relu', input_shape=(X_train.shape[1], 1)),
        MaxPooling1D(2),
        Conv1D(64, 3, activation='relu'),
        MaxPooling1D(2),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')  
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',  
                  metrics=['accuracy'])
    return model

# Validação cruzada
kfold = KFold(n_splits=5, shuffle=True, random_state=seed)
cross_val_scores = []

for train_index, val_index in kfold.split(X_train):
    X_fold_train, X_fold_val = X_train[train_index], X_train[val_index]
    y_fold_train, y_fold_val = y_train.iloc[train_index], y_train.iloc[val_index]

    model = create_cnn_model()
    history = model.fit(X_fold_train, y_fold_train, epochs=25, batch_size=32, verbose=0, validation_data=(X_fold_val, y_fold_val))

    val_accuracy = history.history['val_accuracy'][-1]
    cross_val_scores.append(val_accuracy)

cross_val_mean = np.mean(cross_val_scores)
cross_val_std = np.std(cross_val_scores)

# Treinamento final da CNN
model = create_cnn_model()
history = model.fit(X_train, y_train, epochs=25, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Avaliação da CNN no conjunto de teste
cnn_loss, cnn_accuracy = model.evaluate(X_test, y_test)

# Exibindo resultados da CNN
train_accuracy = history.history['accuracy'][-1]
val_accuracy = history.history['val_accuracy'][-1]

# Predições da CNN no conjunto de teste
cnn_predictions = model.predict(X_test)
cnn_predictions_classes = np.argmax(cnn_predictions, axis=1)

# Classification report da CNN
cnn_report = classification_report(y_test, cnn_predictions_classes, target_names=target_names, zero_division=1, digits=2)

c:\Users\ana_v\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/25
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.3169 - loss: 3.1022 - val_accuracy: 0.3039 - val_loss: 1.5400
Epoch 2/25
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3407 - loss: 1.5188 - val_accuracy: 0.5196 - val_loss: 1.4403
Epoch 3/25
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5345 - loss: 1.2927 - val_accuracy: 0.5196 - val_loss: 1.2385
Epoch 4/25
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5731 - loss: 1.0846 - val_accuracy: 0.5392 - val_loss: 1.1537
Epoch 5/25
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6216 - loss: 0.9843 - val_accuracy: 0.6176 - val_loss: 1.1171
Epoch 6/25
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6770 - loss: 0.8087 - val_accuracy: 0.5980 - val_loss: 1.1156
Epoch 7/25
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7114 - loss: 0.7211 - val_accuracy: 0.6078 - val_loss: 1.0662
Epoch 8/25
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7476 - loss: 0.6378 - val_accuracy: 0.6471 - val_loss

In [12]:
# Resultados
print(f'Acurácia média na validação cruzada (CNN): {cross_val_mean:.4f} ± {cross_val_std:.4f}')
print(f'Acurácia (Teste): {cnn_accuracy:.4f}')
print(f'Classification Report CNN:\n{cnn_report}')
print("Matriz de Confusão:\n", confusion_matrix(y_test, cnn_predictions_classes))

Acurácia média na validação cruzada (CNN): 0.6328 ± 0.0605
Acurácia (Teste): 0.6373
Classification Report CNN:
              precision    recall  f1-score   support

           0       0.54      0.79      0.64        34
           1       0.50      0.50      0.50         8
           2       0.85      0.70      0.77        40
           3       0.50      0.31      0.38        13
           4       0.67      0.29      0.40         7

    accuracy                           0.64       102
   macro avg       0.61      0.52      0.54       102
weighted avg       0.66      0.64      0.63       102

Matriz de Confusão:
 [[27  2  3  2  0]
 [ 2  4  1  0  1]
 [ 9  1 28  2  0]
 [ 8  0  1  4  0]
 [ 4  1  0  0  2]]
